In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ReadAllFilesInFolder").getOrCreate()
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
# Thêm các thư viện Spark cần thiết

spark = SparkSession.builder.getOrCreate()

In [2]:
data_neg = spark.read.text("D:/BinhLuan/data_train/train/neg").limit(5000)
data_pos = spark.read.text("D:/BinhLuan/data_train/train/pos").limit(5000)

data_neg_bage = data_neg.withColumn("label", lit("neg"))
data_pos_bage = data_pos.withColumn("label", lit("pos"))

data = data_neg_bage.union(data_pos_bage)

data.show()

+--------------------+-----+
|               value|label|
+--------------------+-----+
|ESCAPE_GAME : CÁI...|  neg|
|Ít khi mình phàn_...|  neg|
|Vừa đi ăn buffet ...|  neg|
|Thứ nhất , về mặt...|  neg|
|Thứ_hai , về mặt ...|  neg|
|Thứ_ba , về chất_...|  neg|
|Tương_ớt , tương ...|  neg|
|Lúc mình về , thấ...|  neg|
|Hôm_nay đi ăn bò ...|  neg|
|Chỉ có chuyện này...|  neg|
|Rồi đang tung_tẩy...|  neg|
|Rồi bạn ý chạy ra...|  neg|
|Đang ' nói_chuyện...|  neg|
|Mình cũng có nói ...|  neg|
|Rất xin_lỗi anh s...|  neg|
|Nếu em không nhầm...|  neg|
|Mình mà là con_tr...|  neg|
|Mình thật_sự muốn...|  neg|
|Số_là tôi và các ...|  neg|
|Tất_nhiên hội chú...|  neg|
+--------------------+-----+
only showing top 20 rows



In [3]:
data.describe()

DataFrame[summary: string, value: string, label: string]

In [4]:
data.describe().show()

+-------+--------------------+-----+
|summary|               value|label|
+-------+--------------------+-----+
|  count|               10000|10000|
|   mean|                null| null|
| stddev|                null| null|
|    min|! ! ! Sẽ quay lại...|  neg|
|    max|😑 điểm trừ là cơ...|  pos|
+-------+--------------------+-----+



In [5]:
from pyspark.sql.functions import lower

# Chuyển đổi cột "text" thành chữ thường
data = data.select(lower(data.value).alias("value"),data.label)
data.show()

+--------------------+-----+
|               value|label|
+--------------------+-----+
|escape_game : cái...|  neg|
|ít khi mình phàn_...|  neg|
|vừa đi ăn buffet ...|  neg|
|thứ nhất , về mặt...|  neg|
|thứ_hai , về mặt ...|  neg|
|thứ_ba , về chất_...|  neg|
|tương_ớt , tương ...|  neg|
|lúc mình về , thấ...|  neg|
|hôm_nay đi ăn bò ...|  neg|
|chỉ có chuyện này...|  neg|
|rồi đang tung_tẩy...|  neg|
|rồi bạn ý chạy ra...|  neg|
|đang ' nói_chuyện...|  neg|
|mình cũng có nói ...|  neg|
|rất xin_lỗi anh s...|  neg|
|nếu em không nhầm...|  neg|
|mình mà là con_tr...|  neg|
|mình thật_sự muốn...|  neg|
|số_là tôi và các ...|  neg|
|tất_nhiên hội chú...|  neg|
+--------------------+-----+
only showing top 20 rows



In [6]:
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def remove_special_chars(text):
    pattern =  r"[^a-zA-Z0-9_àáảãạăắằẵặẳâấầẩẫậđèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữựỳýỷỹỵ\sđĐ\s]+"
    clean_text = re.sub(pattern, "", text)
    return clean_text

remove_special_chars_udf = udf(remove_special_chars, StringType())

data = data.withColumn("value", remove_special_chars_udf("value"))
data.show()

+--------------------+-----+
|               value|label|
+--------------------+-----+
|escape_game  cái_...|  neg|
|ít khi mình phàn_...|  neg|
|vừa đi ăn buffet ...|  neg|
|thứ nhất  về mặt ...|  neg|
|thứ_hai  về mặt p...|  neg|
|thứ_ba  về chất_l...|  neg|
|tương_ớt  tương c...|  neg|
|lúc mình về  thấy...|  neg|
|hôm_nay đi ăn bò ...|  neg|
|chỉ có chuyện này...|  neg|
|rồi đang tung_tẩy...|  neg|
|rồi bạn ý chạy ra...|  neg|
|đang  nói_chuyện ...|  neg|
|mình cũng có nói ...|  neg|
|rất xin_lỗi anh s...|  neg|
|nếu em không nhầm...|  neg|
|mình mà là con_tr...|  neg|
|mình thật_sự muốn...|  neg|
|số_là tôi và các ...|  neg|
|tất_nhiên hội chú...|  neg|
+--------------------+-----+
only showing top 20 rows



In [7]:
from pyspark.sql.functions import trim

# Xóa khoảng trắng dư thừa trong cột "value"
data = data.withColumn("value", trim(data["value"]))

# Loại bỏ các hàng rỗng
data = data.na.drop()

# In kết quả
data.show()

+--------------------+-----+
|               value|label|
+--------------------+-----+
|escape_game  cái_...|  neg|
|ít khi mình phàn_...|  neg|
|vừa đi ăn buffet ...|  neg|
|thứ nhất  về mặt ...|  neg|
|thứ_hai  về mặt p...|  neg|
|thứ_ba  về chất_l...|  neg|
|tương_ớt  tương c...|  neg|
|lúc mình về  thấy...|  neg|
|hôm_nay đi ăn bò ...|  neg|
|chỉ có chuyện này...|  neg|
|rồi đang tung_tẩy...|  neg|
|rồi bạn ý chạy ra...|  neg|
|đang  nói_chuyện ...|  neg|
|mình cũng có nói ...|  neg|
|rất xin_lỗi anh s...|  neg|
|nếu em không nhầm...|  neg|
|mình mà là con_tr...|  neg|
|mình thật_sự muốn...|  neg|
|số_là tôi và các ...|  neg|
|tất_nhiên hội chú...|  neg|
+--------------------+-----+
only showing top 20 rows



In [8]:
from pyspark.ml.feature import Tokenizer, HashingTF, IDF

#Tách câu thành các token (Tokenization) 
tokenizer = Tokenizer(inputCol="value", outputCol="words")
df = tokenizer.transform(data)

df.show()

+--------------------+-----+--------------------+
|               value|label|               words|
+--------------------+-----+--------------------+
|escape_game  cái_...|  neg|[escape_game, , c...|
|ít khi mình phàn_...|  neg|[ít, khi, mình, p...|
|vừa đi ăn buffet ...|  neg|[vừa, đi, ăn, buf...|
|thứ nhất  về mặt ...|  neg|[thứ, nhất, , về,...|
|thứ_hai  về mặt p...|  neg|[thứ_hai, , về, m...|
|thứ_ba  về chất_l...|  neg|[thứ_ba, , về, ch...|
|tương_ớt  tương c...|  neg|[tương_ớt, , tươn...|
|lúc mình về  thấy...|  neg|[lúc, mình, về, ,...|
|hôm_nay đi ăn bò ...|  neg|[hôm_nay, đi, ăn,...|
|chỉ có chuyện này...|  neg|[chỉ, có, chuyện,...|
|rồi đang tung_tẩy...|  neg|[rồi, đang, tung_...|
|rồi bạn ý chạy ra...|  neg|[rồi, bạn, ý, chạ...|
|đang  nói_chuyện ...|  neg|[đang, , nói_chuy...|
|mình cũng có nói ...|  neg|[mình, cũng, có, ...|
|rất xin_lỗi anh s...|  neg|[rất, xin_lỗi, an...|
|nếu em không nhầm...|  neg|[nếu, em, không, ...|
|mình mà là con_tr...|  neg|[mình, mà, là, co...|


In [9]:
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

#Loại bỏ Stop Word
#Đọc file stop word tiếng việt có sẵn
def read_stopwords(stopword_file):
    with open(stopword_file, 'r', encoding='utf-8') as f:
        stopwords = [line.strip() for line in f]
    return stopwords

# loại bỏ stop word dựa trên dữ liệu stop word tiếng việt ở trên
def remove_stopwords_vn(words, stopwords):
    from underthesea import word_tokenize
    tokens = word_tokenize(words)
    return [w for w in tokens if w not in stopwords]

# Load stop word file
stopword_file = 'vietnamese-stopwords.txt'
stopwords = read_stopwords(stopword_file)

# Define UDF
remove_stopwords_udf = udf(lambda x: remove_stopwords_vn(x, stopwords), ArrayType(StringType()))

#Xóa stop word
remover = StopWordsRemover(inputCol='words', outputCol='filtered_tokens', stopWords=stopwords)
df1 = remover.transform(df).drop('words')
df2 = df1.select('value', 'label', remove_stopwords_udf('value').alias('tokens'))
df2.show()

+--------------------+-----+--------------------+
|               value|label|              tokens|
+--------------------+-----+--------------------+
|escape_game  cái_...|  neg|[escape_game, cái...|
|ít khi mình phàn_...|  neg|[phàn_nàn, chất_l...|
|vừa đi ăn buffet ...|  neg|[đi, buffet, tối,...|
|thứ nhất  về mặt ...|  neg|[mặt, không_gian,...|
|thứ_hai  về mặt p...|  neg|[thứ_hai, mặt, ph...|
|thứ_ba  về chất_l...|  neg|[thứ_ba, chất_lượ...|
|tương_ớt  tương c...|  neg|[tương_ớt, tương,...|
|lúc mình về  thấy...|  neg|[1, trễ, miss, th...|
|hôm_nay đi ăn bò ...|  neg|[hôm_nay, đi, bò,...|
|chỉ có chuyện này...|  neg|[chia_sẻ, nóng, t...|
|rồi đang tung_tẩy...|  neg|[tung_tẩy, đi, ch...|
|rồi bạn ý chạy ra...|  neg|[chạy, chỗ, ghi, ...|
|đang  nói_chuyện ...|  neg|[nói_chuyện, nhân...|
|mình cũng có nói ...|  neg|[kia, hoàn_toàn, ...|
|rất xin_lỗi anh s...|  neg|[xin_lỗi, securit...|
|nếu em không nhầm...|  neg|[nhầm, tuấn, nhân...|
|mình mà là con_tr...|  neg|[con_trai, cục_tí...|


In [10]:
from pyspark.ml.feature import StringIndexer,VectorIndexer

#Gán nhãn cho label
label_indexer = StringIndexer(inputCol="label", outputCol="label_indexer").fit(df2)
data_transformed = label_indexer.transform(df2)
data_transformed.show()

+--------------------+-----+--------------------+-------------+
|               value|label|              tokens|label_indexer|
+--------------------+-----+--------------------+-------------+
|escape_game  cái_...|  neg|[escape_game, cái...|          0.0|
|ít khi mình phàn_...|  neg|[phàn_nàn, chất_l...|          0.0|
|vừa đi ăn buffet ...|  neg|[đi, buffet, tối,...|          0.0|
|thứ nhất  về mặt ...|  neg|[mặt, không_gian,...|          0.0|
|thứ_hai  về mặt p...|  neg|[thứ_hai, mặt, ph...|          0.0|
|thứ_ba  về chất_l...|  neg|[thứ_ba, chất_lượ...|          0.0|
|tương_ớt  tương c...|  neg|[tương_ớt, tương,...|          0.0|
|lúc mình về  thấy...|  neg|[1, trễ, miss, th...|          0.0|
|hôm_nay đi ăn bò ...|  neg|[hôm_nay, đi, bò,...|          0.0|
|chỉ có chuyện này...|  neg|[chia_sẻ, nóng, t...|          0.0|
|rồi đang tung_tẩy...|  neg|[tung_tẩy, đi, ch...|          0.0|
|rồi bạn ý chạy ra...|  neg|[chạy, chỗ, ghi, ...|          0.0|
|đang  nói_chuyện ...|  neg|[nói_chuyện,

In [11]:
hashing_tf = HashingTF(inputCol="tokens", outputCol="HTF_features")

#tính TF 
df3 = hashing_tf.transform(data_transformed)
df3.show()

+--------------------+-----+--------------------+-------------+--------------------+
|               value|label|              tokens|label_indexer|        HTF_features|
+--------------------+-----+--------------------+-------------+--------------------+
|escape_game  cái_...|  neg|[escape_game, cái...|          0.0|(262144,[66983,24...|
|ít khi mình phàn_...|  neg|[phàn_nàn, chất_l...|          0.0|(262144,[158,221,...|
|vừa đi ăn buffet ...|  neg|[đi, buffet, tối,...|          0.0|(262144,[8195,101...|
|thứ nhất  về mặt ...|  neg|[mặt, không_gian,...|          0.0|(262144,[14793,15...|
|thứ_hai  về mặt p...|  neg|[thứ_hai, mặt, ph...|          0.0|(262144,[4686,750...|
|thứ_ba  về chất_l...|  neg|[thứ_ba, chất_lượ...|          0.0|(262144,[402,8195...|
|tương_ớt  tương c...|  neg|[tương_ớt, tương,...|          0.0|(262144,[1875,351...|
|lúc mình về  thấy...|  neg|[1, trễ, miss, th...|          0.0|(262144,[68,11592...|
|hôm_nay đi ăn bò ...|  neg|[hôm_nay, đi, bò,...|          0.0|(2

In [12]:
idf = IDF(inputCol="HTF_features", outputCol="features")

#tính ITF 
idfModel = idf.fit(df3)
                   
df4 = idfModel.transform(df3)
df4.show()

+--------------------+-----+--------------------+-------------+--------------------+--------------------+
|               value|label|              tokens|label_indexer|        HTF_features|            features|
+--------------------+-----+--------------------+-------------+--------------------+--------------------+
|escape_game  cái_...|  neg|[escape_game, cái...|          0.0|(262144,[66983,24...|(262144,[66983,24...|
|ít khi mình phàn_...|  neg|[phàn_nàn, chất_l...|          0.0|(262144,[158,221,...|(262144,[158,221,...|
|vừa đi ăn buffet ...|  neg|[đi, buffet, tối,...|          0.0|(262144,[8195,101...|(262144,[8195,101...|
|thứ nhất  về mặt ...|  neg|[mặt, không_gian,...|          0.0|(262144,[14793,15...|(262144,[14793,15...|
|thứ_hai  về mặt p...|  neg|[thứ_hai, mặt, ph...|          0.0|(262144,[4686,750...|(262144,[4686,750...|
|thứ_ba  về chất_l...|  neg|[thứ_ba, chất_lượ...|          0.0|(262144,[402,8195...|(262144,[402,8195...|
|tương_ớt  tương c...|  neg|[tương_ớt, tương,.

In [13]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, regParam=0.01, featuresCol="features", labelCol="label_indexer", predictionCol="prediction", probabilityCol="probability", rawPredictionCol="rawPrediction")

# tạo pipeline để thực hiện các bước xử lý dữ liệu và huấn luyện mô hình
pipeline = Pipeline(stages=[tokenizer, remover, hashing_tf, idf, lr])

In [14]:
#chia tập dữ liệu thành train, test theo tỷ lệ 70:30
#(train_data, test_data) = data_transformed.randomSplit([0.7, 0.3], seed=12345)
#chia tập dữ liệu thành train, test theo tỷ lệ 80:20
(train_data, test_data) = data_transformed.randomSplit([0.8, 0.2], seed=12345)

In [15]:
model = pipeline.fit(train_data)

In [16]:
predictions = model.transform(test_data)

In [17]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#tính toán độ chính xác
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label_indexer", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("Accuracy = %g" % (accuracy))

Accuracy = 0.751246


In [18]:
predictions.filter(predictions.label_indexer == 1.0).select('label', 'label_indexer', 'prediction').show()

+-----+-------------+----------+
|label|label_indexer|prediction|
+-----+-------------+----------+
|  pos|          1.0|       0.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       0.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       0.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
+-----+-------------+----------+
only showing top 20 rows



In [19]:
predictions.filter(predictions.label_indexer == 0.0).select('label', 'label_indexer', 'prediction').show()

+-----+-------------+----------+
|label|label_indexer|prediction|
+-----+-------------+----------+
|  neg|          0.0|       0.0|
|  neg|          0.0|       0.0|
|  neg|          0.0|       0.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       0.0|
|  neg|          0.0|       0.0|
|  neg|          0.0|       0.0|
|  neg|          0.0|       0.0|
|  neg|          0.0|       0.0|
|  neg|          0.0|       0.0|
|  neg|          0.0|       0.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       0.0|
|  neg|          0.0|       0.0|
|  neg|          0.0|       0.0|
|  neg|          0.0|       0.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       0.0|
|  neg|          0.0|       0.0|
|  neg|          0.0|       1.0|
+-----+-------------+----------+
only showing top 20 rows

